In [1]:
### Formulated by Rohit Ghosh ###
import pyfesom2 as pf
import xarray as xr
import numpy as np

In [2]:
mesh_path='/work/ab0995/a270088/meshes/NG5/'
diag = xr.open_dataset((mesh_path+'fesom.mesh.diag.nc')) 
ddx = diag['gradient_sca_x']
ddy = diag['gradient_sca_y']
elem = (diag['elements']-1).T  

In [ ]:
%%time

# Define the years and months
years = ['1971']  # Example list of years
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']  # List of months

# Loop over years and months
for year in years:
    for mon in months:
        data_path='/work/bm1344/a270228/EERIE_NextG_Hackathon/IFS-FESOM_CONTROL-1950/tco1279-NG5/{year}/FESOM/native/daily/'

        # Use string formatting to replace {year} in the data_path
        data_path = data_path.format(year=year)

        sst = xr.open_mfdataset((data_path+f'fesom_sst_{year}{mon}*-{year}{mon}*_daily-NG5.nc'),chunks={'time':40,'nod2':200})['sst'].astype('float32')
        time= xr.open_mfdataset((data_path+f'fesom_sst_{year}{mon}*-{year}{mon}*_daily-NG5.nc'),chunks={'time':40,'nod2':200})['time'].astype('float32')

        n= time.size

        mesh= pf.load_mesh(mesh_path)

       # Initialize empty lists to store results for each time step
        dt_dx_list = []
        dt_dy_list = []

        for dd in range(n):  # Looping over 0 to 30 for the time dimension
            t = sst[dd, :].compute()

    
            dt_dy = (ddy * t[elem]).sum(dim='n3')
            dt_dx = (ddx * t[elem]).sum(dim='n3') 
    

            ## Divergence at Node ##
            dt_dy = pf.tonodes(dt_dy, mesh)
            dt_dx = pf.tonodes(dt_dx, mesh)
    
            ## Changing dt_dy and dt_dx to Xarray object ##
            dt_dx = xr.DataArray(dt_dx, coords=t.coords, dims=t.dims)
            dt_dy = xr.DataArray(dt_dy, coords=t.coords, dims=t.dims)
    
            # Append div and curl to lists
            dt_dx_list.append(dt_dx)
            dt_dy_list.append(dt_dy)

            # Combine the lists into Xarray DataArrays along the time dimension
            dsst_dx = xr.concat(dt_dx_list, dim='time')
            dsst_dy = xr.concat(dt_dy_list, dim='time')

            # Add variable attributes
            dsst_dx.attrs['long_name'] = 'Gradient of Sea Surface Temperature in the x-direction'
            dsst_dx.attrs['units'] = 'degree Celsius per meter'

            # Create a Dataset containing the DataArray
            dsx = xr.Dataset({'dsst_dx': dsst_dx})

            dsx.to_netcdf(f"/work/bm1344/a270228/gradient/fesom_dSSTdX_{year}{mon}_daily-NG5.nc")

            # Add variable attributes
            dsst_dy.attrs['long_name'] = 'Gradient of Sea Surface Temperature in the y-direction'
            dsst_dy.attrs['units'] = 'degree Celsius per meter'

            # Create a Dataset containing the DataArray
            dsy = xr.Dataset({'dsst_dy': dsst_dy})

            dsy.to_netcdf(f"/work/bm1344/a270228/gradient/fesom_dSSTdY_{year}{mon}_daily-NG5.nc")



/work/ab0995/a270088/meshes/NG5/pickle_mesh_py3_fesom2
The usepickle == True)
The pickle file for FESOM2 exists.
The mesh will be loaded from /work/ab0995/a270088/meshes/NG5/pickle_mesh_py3_fesom2
